In [1]:
# | default_exp routes.auth

In [2]:
# | hide
from nbdev.showdoc import *
import fastcore.test as fctest

In [3]:
# | exporti
from dataclasses import dataclass, field
from typing import Optional, Union
from urllib.parse import urlparse

import httpx

import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.Logger as lg
import domolibrary.client.DomoError as de

# DomoError Classes


In [4]:
# | export
class InvalidCredentialsError(de.DomoError):
    """return invalid credentials sent to API"""

    def __init__(
        self,
        function_name: Optional[str] = None,
        parent_class: str = None,
        status: Optional[int] = None,  # API request status
        message="invalid credentials",
        domo_instance: Optional[str] = None,
    ):
        super().__init__(
            status=status,
            message=message,
            domo_instance=domo_instance,
            function_name=function_name,
            parent_class=parent_class,
        )


class AccountLockedError(de.DomoError):
    """return invalid credentials sent to API"""

    def __init__(
        self,
        function_name: Optional[str] = None,
        status: Optional[int] = None,  # API request status
        message="invalid credentials",
        domo_instance: Optional[str] = None,
        parent_class: str = None,
    ):
        super().__init__(
            status=status,
            message=message,
            domo_instance=domo_instance,
            function_name=function_name,
            parent_class=parent_class,
        )


class InvalidAuthTypeError(de.DomoError):
    """return invalid Auth type sent to API"""

    def __init__(
        self,
        required_auth_type: dict = None,
        required_auth_type_ls: list = None,
        function_name: Optional[str] = None,
        parent_class: str = None,
        domo_instance: Optional[str] = None,
    ):
        message = f"This API rquires {required_auth_type.__name__ if required_auth_type else ', '.join([auth_type.__name__ for auth_type in required_auth_type_ls])}"

        super().__init__(
            message=message,
            domo_instance=domo_instance,
            function_name=function_name,
            parent_class=parent_class,
        )


class InvalidInstanceError(de.DomoError):
    """return if invalid domo_instance sent to API"""

    def __init__(
        self,
        function_name: Optional[str] = None,
        parent_class: str = None,
        status: Optional[int] = None,
        message="invalid instance",
        domo_instance: Optional[str] = None,
    ):
        super().__init__(
            status=status,
            message=message,
            domo_instance=domo_instance,
            parent_class=parent_class,
            function_name=function_name,
        )


class NoAccessTokenReturned(de.DomoError):
    def __init__(
        self,
        function_name: Optional[str] = None,
        status: Optional[int] = None,
        message: str = "No AccessToken returned",
        domo_instance: Optional[str] = None,
        parent_class: str = None,
    ):
        super().__init__(
            status=status,
            message=message,
            domo_instance=domo_instance,
            function_name=function_name,
            parent_class=parent_class,
        )

# Authentication Routes

Stand alone functions for users who prefer a functional programming approach

## Full Auth Route - username and password authentication


In [5]:
# | export

async def get_full_auth(
    domo_instance: str,  # domo_instance.domo.com
    domo_username: str,  # email address
    domo_password: str,
    session: Optional[httpx.AsyncClient] = None,
    debug_api: bool = False,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    """uses username and password authentication to retrieve a full_auth access token"""

    is_close_session = False

    if not session:
        is_close_session = True
        session = httpx.AsyncClient()

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    tokenHeaders = {"Content-Type": "application/json"}

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password,
    }

    if debug_api:
        print(body, url)

    res = await session.request(method="POST", url=url, headers=tokenHeaders, json=body)

    if is_close_session:
        await session.aclose()

    traceback_details = lg.get_traceback()

    res = rgd.ResponseGetData._from_httpx_response(
        res, traceback_details=traceback_details, parent_class=parent_class
    )

    if res.is_success and res.response.get("reason", None):
        if res.response.get("reason") == "INVALID_CREDENTIALS":
            res.is_success = False
            raise InvalidCredentialsError(
                function_name=res.traceback_details.function_name,
                parent_class=parent_class,
                status=res.status,
                message=res.response["reason"],
                domo_instance=domo_instance,
            )
        if res.response.get("reason") == "ACCOUNT_LOCKED":
            res.is_success = False
            raise AccountLockedError(
                function_name=res.traceback_details.function_name,
                parent_class=parent_class,
                status=res.status,
                message=str(res.response.get("reason")),
                domo_instance=self.domo_instance,
            )

        if res.response == {} or res.response == "":  # no access token
            res.is_success = False
            raise NoAccessTokenReturned(
                function_name=res.traceback_details.function_name,
                parent_class=parent_class,
                status=res.status,
                domo_instance=self.domo_instance,
            )

    if res.status == 403 and res.response == "Forbidden":
        res.is_success = False
        raise InvalidInstanceError(
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
            status=res.status,
            message=res.response,
            domo_instance=domo_instance,
        )

    if not res.is_success:
        raise InvalidCredentialsError(
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
            status=res.status,
            message=res.response["reason"],
            domo_instance=domo_instance,
        )

    return res


#### Sample Implementations of get_full_auth

##### intentional invalid credentials


In [6]:
import os

try:
    await get_full_auth(
        domo_instance="domo-community",
        domo_username="test@test.com",
        domo_password="fake password",
    )
except InvalidCredentialsError as e:
    print(e)


🛑  InvalidCredentialsError 🛑 - functionn: get_full_auth || status 200 || INVALID_CREDENTIALS at domo-community


##### invalid instance


In [7]:
try:
    await get_full_auth(
        domo_instance="test",
        domo_username="fake@test.com",
        domo_password="fake password",
    )
except InvalidInstanceError as e:
    print(e)


🛑  InvalidInstanceError 🛑 - functionn: get_full_auth || status 403 || Forbidden at test


##### valid credentials


In [8]:
res = await get_full_auth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)

assert res.is_success


## Developer Auth Route - client_id and secret authentication


In [9]:
# | export
async def get_developer_auth(
    domo_client_id: str,
    domo_client_secret: str,
    session: Optional[httpx.AsyncClient] = None,
    debug_api: bool = False,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    """
    only use for authenticating against apis documented under developer.domo.com
    """
    is_close_session = False

    if not session:
        is_close_session = True
        session = httpx.AsyncClient(
            auth=httpx.BasicAuth(domo_client_id, domo_client_secret)
        )

    url = "https://api.domo.com/oauth/token?grant_type=client_credentials"

    if debug_api:
        print(url, domo_client_id, domo_client_secret)

    res = await session.request(method="GET", url=url)

    traceback_details = lg.get_traceback()

    if is_close_session:
        await session.aclose()

    res = rgd.ResponseGetData._from_httpx_response(
        res, 
        traceback_details=traceback_details,
        parent_class=parent_class
    )

    if res.status == 401 and res.response == "Unauthorized":
        res.is_success = False
        raise InvalidCredentialsError(
            function_name=res.traceback_details.function_name,
            status=res.status,
            message=res.response,
        )

    return res


#### Sample Implementations of get_developer_auth

The 401 response is expected because we are using invalid credentials


In [10]:
try:
    await get_developer_auth(domo_client_id="test_id", domo_client_secret="test_secret")

except InvalidCredentialsError as e:
    print(e)


🛑  InvalidCredentialsError 🛑 - functionn: get_developer_auth || status 401 || Unauthorized


# Who Am I?


In [11]:
# | export
async def who_am_i(
    auth_header: dict,
    domo_instance: str,  # <domo_instance>.domo.com
    session: httpx.AsyncClient = None,
    parent_class :str = None,
    debug_num_stacks_to_drop = 0,
    debug_api: bool = False,
    return_raw: bool = False
):
    """
    will attempt to validate against the 'me' API.
    This is the same authentication test the Domo Java CLI uses.
    """

    is_close_session = False

    if not session:
        is_close_session = True
        session = httpx.AsyncClient()

    url = f"https://{domo_instance}.domo.com/api/content/v2/users/me"

    if debug_api:
        print(url, auth_header)

    res = await session.request(method="GET", headers=auth_header, url=url)

    if is_close_session:
        await session.aclose()
    
    if return_raw:
        return res
    
    traceback_details = lg.get_traceback(num_stacks_to_drop= debug_num_stacks_to_drop)

    res = rgd.ResponseGetData._from_httpx_response(res, traceback_details= traceback_details, parent_class= parent_class)

    if res.status == 401 and res.response == "Unauthorized":
        res.is_sucess = False

        raise InvalidCredentialsError(
            function_name = res.traceback_details.function_name,
            parent_class = parent_class,
            status=res.status,
            message=res.response,
            domo_instance=domo_instance,
        )

    return res


#### Sample implementation of `who_am_i`

##### test developer token


In [12]:
import os

await who_am_i(
    domo_instance="domo-community",
    auth_header={
        "x-domo-developer-token": os.environ["DOMO_DOJO_ACCESS_TOKEN"]},
    debug_api=False,
    return_raw=False
)


ResponseGetData(status=200, response={'id': 1893952720, 'invitorUserId': 587894148, 'displayName': 'Jae Wilson1', 'department': 'Business Improvement', 'userName': 'jae@onyxreporting.com', 'emailAddress': 'jae@onyxreporting.com', 'avatarKey': 'c605f478-0cd2-4451-9fd4-d82090b71e66', 'accepted': True, 'userType': 'USER', 'modified': 1686313430521, 'created': 1588960518, 'active': True, 'pending': False, 'systemUser': False, 'anonymous': False}, is_success=True, parent_class=None, traceback_details=TracebackDetails(function_name='who_am_i', file_name='/tmp/ipykernel_4671/1424343251.py', function_trail='<module> -> who_am_i', traceback_stack=[<FrameSummary file /tmp/ipykernel_4671/1753996340.py, line 3 in <module>>, <FrameSummary file /tmp/ipykernel_4671/1424343251.py, line 35 in who_am_i>], parent_class=None))

##### test full_auth token


In [13]:
res = await get_full_auth(
    domo_instance="domo-community",
    domo_username=os.environ["DOMO_USERNAME"],
    domo_password=os.environ["DOJO_PASSWORD"],
)

session_token = res.response.get("sessionToken")

await who_am_i(
    domo_instance="domo-community", auth_header={"x-domo-authentication": session_token}
)


ResponseGetData(status=200, response={'id': 1893952720, 'invitorUserId': 587894148, 'displayName': 'Jae Wilson1', 'department': 'Business Improvement', 'userName': 'jae@onyxreporting.com', 'emailAddress': 'jae@onyxreporting.com', 'avatarKey': 'c605f478-0cd2-4451-9fd4-d82090b71e66', 'accepted': True, 'userType': 'USER', 'modified': 1686313430521, 'created': 1588960518, 'active': True, 'pending': False, 'systemUser': False, 'anonymous': False}, is_success=True, parent_class=None, traceback_details=TracebackDetails(function_name='who_am_i', file_name='/tmp/ipykernel_4671/1424343251.py', function_trail='<module> -> who_am_i', traceback_stack=[<FrameSummary file /tmp/ipykernel_4671/33241985.py, line 9 in <module>>, <FrameSummary file /tmp/ipykernel_4671/1424343251.py, line 35 in who_am_i>], parent_class=None))

In [14]:
# | hide
import nbdev

nbdev.nbdev_export()